In [18]:
####http://stackoverflow.com/questions/8376691/how-to-remove-hashtag-user-link-of-a-tweet-using-regular-expression
import re,string

def strip_links(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    return text

def strip_all_entities(text):
    ###strip only @mentions and not #hashtags!
    entity_prefixes = ['@']
#     entity_prefixes = ['@','#']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

tests = [
    "@peter I really love that shirt at #Macy. http://bet.ly//WjdiW4",
    "@shawn Titanic tragedy could have been prevented Economic Times: Telegraph.co.ukTitanic tragedy could have been preve... http://bet.ly/tuN2wx",
    "I am at Starbucks http://4sh.com/samqUI (7419 3rd ave, at 75th, Brooklyn)",
]
for t in tests:
    print strip_all_entities(strip_links(t))

I really love that shirt at
Titanic tragedy could have been prevented Economic Times Telegraph co ukTitanic tragedy could have been preve
I am at Starbucks 7419 3rd ave at 75th Brooklyn


In [19]:
filename = ['pos_examples_happy.txt', 'neg_examples_sad.txt','pos_examples_PosSentiment.txt', 'neg_examples_NegSentiment.txt']
# filename = ['pos_examples_happy.txt']
for file_ in filename:
    print file_
    target = open('data/nohashtags'+file_, 'w')
    count = 0

    with open('data/'+file_, 'r') as f:
        pos_tweets = f.readlines()
        for tweet in pos_tweets:
            count += 1
            target.write(strip_all_entities(strip_links(tweet)))
            target.write('\n')

    print "Done: ", count

pos_examples_happy.txt
Done:  17740
neg_examples_sad.txt
Done:  17740
pos_examples_PosSentiment.txt
Done:  100000
neg_examples_NegSentiment.txt
Done:  100000


In [11]:
###test on preprocessed data

##### load training data
import random
import numpy as np
import pandas as pd

# suffix = ""
suffix = "preprocessed_"
###read tagged sentiment Tweets
with open('data/'+suffix+'pos_examples_PosSentiment.txt', 'r') as f:
    pos_tweets = f.readlines()

with open('data/'+suffix+'neg_examples_NegSentiment.txt', 'r') as f:
    neg_tweets = f.readlines()
    
class_label = 0
train_pos = pd.DataFrame(pos_tweets, columns = ['text'])
X0 = train_pos.text
y0 = np.full(shape=len(X0), fill_value=class_label)

###create random 80-20 split
rows = random.sample(X0.index, int(0.8*len(X0)))
X0_80 = X0.ix[rows]
X0_20 = X0.drop(rows)
y0_80 = y0[:int(0.8*len(X0))]
y0_20 = y0[int(0.8*len(X0)):]
###

class_label = 1
train_neg = pd.DataFrame(neg_tweets, columns = ['text'])
X1 = train_neg.text
y1 = np.full(shape=len(X1), fill_value=class_label)

###create random 80-20 split
rows = random.sample(X1.index, int(0.8*len(X1)))
X1_80 = X1.ix[rows]
X1_20 = X1.drop(rows)
y1_80 = y1[:int(0.8*len(X1))]
y1_20 = y1[int(0.8*len(X1)):]
###

X_train = pd.concat((X0_80,X1_80))
y_train = np.concatenate((y0_80, y1_80))

X_holdout = pd.concat((X0_20,X1_20))
y_holdout = np.concatenate((y0_20, y1_20))

### load testing data

###read tagged sentiment Tweets
with open('data/'+suffix+'pos_examples_happy.txt', 'r') as f:
    pos_tweets = f.readlines()

with open('data/'+suffix+'neg_examples_sad.txt', 'r') as f:
    neg_tweets = f.readlines()
    
import numpy as np
class_label = 0
train_pos = pd.DataFrame(pos_tweets, columns = ['text'])
X0 = train_pos.text
y0 = np.full(shape=len(X0), fill_value=class_label)

class_label = 1
train_neg = pd.DataFrame(neg_tweets, columns = ['text'])
X1 = train_neg.text
y1 = np.full(shape=len(X1), fill_value=class_label)

X_test = pd.concat((X0,X1))
y_test = np.concatenate((y0, y1))

###
print "Loading data = COMPLETE!"

Loading data = COMPLETE!


In [17]:
##Generate top 5000 ngrams and select random 50
###print top 5000 features 
import nltk
from sklearn.feature_extraction.text import CountVectorizer

##vectorizer arguments blah!

tokenizer=None#word_tokenize
stop_words=nltk.corpus.stopwords.words("english")#None
stop_words.append('rt')
ngram_range=(1, 3)
lowercase=True
max_features=5000
binary=False
dtype=np.float64

###create vectorizer
vectorizer = CountVectorizer(decode_error="ignore",
#                                tokenizer=tokenizer,
                               stop_words=stop_words,
                               ngram_range=ngram_range,
                               lowercase=lowercase,
                               binary=binary,
                               dtype=dtype,
                               max_features=max_features)

train_fit = vectorizer.fit_transform(X_train)

vocab = vectorizer.vocabulary_
arr = (train_fit.toarray().sum(axis=0))

with open("top5000.txt", "w") as f:
    for key in vocab:
        try:
            f.write(str(key) + ',' + str(arr[vocab[key]]) + "\n")
        except:
            continue
print "Done getting top 5000 ngrams!"

Done getting top 5000 ngrams!


In [14]:
### build pipeline; fit train; predict on holdout
import nltk
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

##vectorizer arguments blah!

tokenizer=None#word_tokenize
stop_words=nltk.corpus.stopwords.words("english")#None
ngram_range=(1, 3)
lowercase=True
max_features=5000
binary=False
dtype=np.float64
    
###create vectorizer
vectorizer = CountVectorizer(decode_error="ignore",
#                                tokenizer=tokenizer,
                               stop_words=stop_words,
                               ngram_range=ngram_range,
                               lowercase=lowercase,
                               binary=binary,
                               dtype=dtype,
                               max_features=max_features)

###create pipeline
classifier = "MNB"
text_clf = Pipeline([
                     ('vect', vectorizer),
                     ('clf', MultinomialNB(alpha=0.1)),
#                     ('clf', LinearSVC(C=1.0)),
#         ("clf", RandomForestClassifier(n_estimators=100, max_features="auto", criterion="entropy", n_jobs=-1)),
                    ])

###fit training data
text_clf = text_clf.fit(X_train, y_train)

### predict on test data
predicted = text_clf.predict(X_holdout)
# np.mean(predicted == y_test)

###get performance metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
accuracy = accuracy_score(y_holdout,predicted)
#         accuracy = np.mean(y_CVtest == predicted)
precision, recall, fscore, sup = precision_recall_fscore_support(y_holdout, predicted, average='binary', pos_label=0)
print classifier, accuracy, precision, recall, fscore# np.mean(predicted == y_test)
###
###create pipeline
classifier = "SVC"
text_clf = Pipeline([
                     ('vect', vectorizer),
#                      ('clf', MultinomialNB(alpha=0.1)),
                    ('clf', LinearSVC(C=1.0)),
#         ("clf", RandomForestClassifier(n_estimators=100, max_features="auto", criterion="entropy", n_jobs=-1)),
                    ])

###fit training data
text_clf = text_clf.fit(X_train, y_train)

### predict on test data
predicted = text_clf.predict(X_holdout)
# np.mean(predicted == y_test)

###get performance metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
accuracy = accuracy_score(y_holdout,predicted)
#         accuracy = np.mean(y_CVtest == predicted)
precision, recall, fscore, sup = precision_recall_fscore_support(y_holdout, predicted, average='binary', pos_label=0)
print classifier, accuracy, precision, recall, fscore# np.mean(predicted == y_test)
###
###create pipeline
classifier = "RF"
text_clf = Pipeline([
                     ('vect', vectorizer),
#                      ('clf', MultinomialNB(alpha=0.1)),
#                     ('clf', LinearSVC(C=1.0)),
        ("clf", RandomForestClassifier(n_estimators=100, max_features="auto", criterion="entropy", n_jobs=-1)),
                    ])

###fit training data
text_clf = text_clf.fit(X_train, y_train)

### predict on test data
predicted = text_clf.predict(X_holdout)
# np.mean(predicted == y_test)

###get performance metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
accuracy = accuracy_score(y_holdout,predicted)
#         accuracy = np.mean(y_CVtest == predicted)
precision, recall, fscore, sup = precision_recall_fscore_support(y_holdout, predicted, average='binary', pos_label=0)
print classifier, accuracy, precision, recall, fscore# np.mean(predicted == y_test)
###


print "Prediction on Holdout (20%) = DONE!"

MNB 0.735725 0.732390200621 0.7429 0.737607665004
SVC 0.742825 0.720259422196 0.79405 0.755356845585
RF 0.73985 0.729785399502 0.76175 0.745425188375
Prediction on Holdout (20%) = DONE!


In [15]:
### build pipeline; fit train; predict on test
import nltk
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

##vectorizer arguments blah!

tokenizer=None#word_tokenize
stop_words=nltk.corpus.stopwords.words("english")#None
ngram_range=(1, 3)
lowercase=True
max_features=5000
binary=False
dtype=np.float64
    
###create vectorizer
vectorizer = CountVectorizer(decode_error="ignore",
#                                tokenizer=tokenizer,
                               stop_words=stop_words,
                               ngram_range=ngram_range,
                               lowercase=lowercase,
                               binary=binary,
                               dtype=dtype,
                               max_features=max_features)

###create pipeline
classifier = "MNB"
text_clf = Pipeline([
                     ('vect', vectorizer),
                     ('clf', MultinomialNB(alpha=0.1)),
#                     ('clf', LinearSVC(C=1.0)),
#         ("clf", RandomForestClassifier(n_estimators=100, max_features="auto", criterion="entropy", n_jobs=-1)),
                    ])

###fit training data
text_clf = text_clf.fit(X_train, y_train)

### predict on test data
predicted = text_clf.predict(X_test)
# np.mean(predicted == y_test)

###get performance metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
accuracy = accuracy_score(y_test,predicted)
#         accuracy = np.mean(y_CVtest == predicted)
precision, recall, fscore, sup = precision_recall_fscore_support(y_test, predicted, average='binary', pos_label=0)
print classifier, accuracy, precision, recall, fscore# np.mean(predicted == y_test)
###
###create pipeline
classifier = "SVC"
text_clf = Pipeline([
                     ('vect', vectorizer),
#                      ('clf', MultinomialNB(alpha=0.1)),
                    ('clf', LinearSVC(C=1.0)),
#         ("clf", RandomForestClassifier(n_estimators=100, max_features="auto", criterion="entropy", n_jobs=-1)),
                    ])

###fit training data
text_clf = text_clf.fit(X_train, y_train)

### predict on test data
predicted = text_clf.predict(X_test)
# np.mean(predicted == y_test)

###get performance metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
accuracy = accuracy_score(y_test,predicted)
#         accuracy = np.mean(y_CVtest == predicted)
precision, recall, fscore, sup = precision_recall_fscore_support(y_test, predicted, average='binary', pos_label=0)
print classifier, accuracy, precision, recall, fscore# np.mean(predicted == y_test)
###
###create pipeline
classifier = "RF"
text_clf = Pipeline([
                     ('vect', vectorizer),
#                      ('clf', MultinomialNB(alpha=0.1)),
#                     ('clf', LinearSVC(C=1.0)),
        ("clf", RandomForestClassifier(n_estimators=100, max_features="auto", criterion="entropy", n_jobs=-1)),
                    ])

###fit training data
text_clf = text_clf.fit(X_train, y_train)

### predict on test data
predicted = text_clf.predict(X_test)
# np.mean(predicted == y_test)

###get performance metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
accuracy = accuracy_score(y_test,predicted)
#         accuracy = np.mean(y_CVtest == predicted)
precision, recall, fscore, sup = precision_recall_fscore_support(y_test, predicted, average='binary', pos_label=0)
print classifier, accuracy, precision, recall, fscore# np.mean(predicted == y_test)
###


print "Prediction on Test (#happy, #sad) = DONE!"

MNB 0.952418581024 0.93775978998 0.969029049395 0.953138028795
SVC 0.961985663487 0.964007947772 0.959703854414 0.961851086125
RF 0.96562623469 0.978064066852 0.952526280095 0.965126266964
Prediction on Test (#happy, #sad) = DONE!


In [23]:
###test on preprocessed data

##### load training data
import random
import numpy as np
import pandas as pd

# suffix = ""
# suffix = "nohashtags"
suffix = "preprocessed_"
###read tagged sentiment Tweets
with open('data/'+suffix+'pos_examples_PosSentiment.txt', 'r') as f:
    pos_tweets = f.readlines()

with open('data/'+suffix+'neg_examples_NegSentiment.txt', 'r') as f:
    neg_tweets = f.readlines()
    
class_label = 0
train_pos = pd.DataFrame(pos_tweets, columns = ['text'])
X0 = train_pos.text
y0 = np.full(shape=len(X0), fill_value=class_label)

###create random 80-20 split
rows = random.sample(X0.index, int(0.8*len(X0)))
X0_80 = X0.ix[rows]
X0_20 = X0.drop(rows)
y0_80 = y0[:int(0.8*len(X0))]
y0_20 = y0[int(0.8*len(X0)):]
###

class_label = 1
train_neg = pd.DataFrame(neg_tweets, columns = ['text'])
X1 = train_neg.text
y1 = np.full(shape=len(X1), fill_value=class_label)

###create random 80-20 split
rows = random.sample(X1.index, int(0.8*len(X1)))
X1_80 = X1.ix[rows]
X1_20 = X1.drop(rows)
y1_80 = y1[:int(0.8*len(X1))]
y1_20 = y1[int(0.8*len(X1)):]
###

X_train = pd.concat((X0_80,X1_80))
y_train = np.concatenate((y0_80, y1_80))

X_holdout = pd.concat((X0_20,X1_20))
y_holdout = np.concatenate((y0_20, y1_20))

### load testing data

###read tagged sentiment Tweets
with open('data/'+suffix+'pos_examples_happy.txt', 'r') as f:
    pos_tweets = f.readlines()

with open('data/'+suffix+'neg_examples_sad.txt', 'r') as f:
    neg_tweets = f.readlines()
    
import numpy as np
class_label = 0
train_pos = pd.DataFrame(pos_tweets, columns = ['text'])
X0 = train_pos.text
y0 = np.full(shape=len(X0), fill_value=class_label)

class_label = 1
train_neg = pd.DataFrame(neg_tweets, columns = ['text'])
X1 = train_neg.text
y1 = np.full(shape=len(X1), fill_value=class_label)

X_test = pd.concat((X0,X1))
y_test = np.concatenate((y0, y1))

###
print "Loading data = COMPLETE!"

Loading data = COMPLETE!


In [21]:
### build pipeline; fit train; predict on test W/O Hashtags!
import nltk
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

##vectorizer arguments blah!

tokenizer=None#word_tokenize
stop_words=nltk.corpus.stopwords.words("english")#None
ngram_range=(1, 3)
lowercase=True
max_features=5000
binary=False
dtype=np.float64
    
###create vectorizer
vectorizer = CountVectorizer(decode_error="ignore",
#                                tokenizer=tokenizer,
                               stop_words=stop_words,
                               ngram_range=ngram_range,
                               lowercase=lowercase,
                               binary=binary,
                               dtype=dtype,
                               max_features=max_features)

###create pipeline
classifier = "MNB"
text_clf = Pipeline([
                     ('vect', vectorizer),
                     ('clf', MultinomialNB(alpha=0.1)),
#                     ('clf', LinearSVC(C=1.0)),
#         ("clf", RandomForestClassifier(n_estimators=100, max_features="auto", criterion="entropy", n_jobs=-1)),
                    ])

###fit training data
text_clf = text_clf.fit(X_train, y_train)

### predict on test data
predicted = text_clf.predict(X_test)
# np.mean(predicted == y_test)

###get performance metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
accuracy = accuracy_score(y_test,predicted)
#         accuracy = np.mean(y_CVtest == predicted)
precision, recall, fscore, sup = precision_recall_fscore_support(y_test, predicted, average='binary', pos_label=0)
print classifier, accuracy, precision, recall, fscore# np.mean(predicted == y_test)
###
###create pipeline
classifier = "SVC"
text_clf = Pipeline([
                     ('vect', vectorizer),
#                      ('clf', MultinomialNB(alpha=0.1)),
                    ('clf', LinearSVC(C=1.0)),
#         ("clf", RandomForestClassifier(n_estimators=100, max_features="auto", criterion="entropy", n_jobs=-1)),
                    ])

###fit training data
text_clf = text_clf.fit(X_train, y_train)

### predict on test data
predicted = text_clf.predict(X_test)
# np.mean(predicted == y_test)

###get performance metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
accuracy = accuracy_score(y_test,predicted)
#         accuracy = np.mean(y_CVtest == predicted)
precision, recall, fscore, sup = precision_recall_fscore_support(y_test, predicted, average='binary', pos_label=0)
print classifier, accuracy, precision, recall, fscore# np.mean(predicted == y_test)
###
###create pipeline
classifier = "RF"
text_clf = Pipeline([
                     ('vect', vectorizer),
#                      ('clf', MultinomialNB(alpha=0.1)),
#                     ('clf', LinearSVC(C=1.0)),
        ("clf", RandomForestClassifier(n_estimators=100, max_features="auto", criterion="entropy", n_jobs=-1)),
                    ])

###fit training data
text_clf = text_clf.fit(X_train, y_train)

### predict on test data
predicted = text_clf.predict(X_test)
# np.mean(predicted == y_test)

###get performance metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
accuracy = accuracy_score(y_test,predicted)
#         accuracy = np.mean(y_CVtest == predicted)
precision, recall, fscore, sup = precision_recall_fscore_support(y_test, predicted, average='binary', pos_label=0)
print classifier, accuracy, precision, recall, fscore# np.mean(predicted == y_test)
###


print "Prediction on Test (#happy, #sad) w/0 hashtags = DONE!"

MNB 0.645518602029 0.598271727131 0.885907553551 0.714217546411
SVC 0.666065388952 0.614496696463 0.891262683202 0.727444214401
RF 0.631313416009 0.592561687925 0.840642615558 0.695131330552
Prediction on Test (#happy, #sad) w/0 hashtags = DONE!


In [ ]:
###test on preprocessed data

##### load training data
import random
import numpy as np
import pandas as pd

# suffix = ""
# suffix = "nohashtags"
suffix = "preprocessed_"
###read tagged sentiment Tweets
with open('data/'+suffix+'pos_examples_PosSentiment.txt', 'r') as f:
    pos_tweets = f.readlines()

with open('data/'+suffix+'neg_examples_NegSentiment.txt', 'r') as f:
    neg_tweets = f.readlines()
    
class_label = 0
train_pos = pd.DataFrame(pos_tweets, columns = ['text'])
X0 = train_pos.text
y0 = np.full(shape=len(X0), fill_value=class_label)

###create random 80-20 split
rows = random.sample(X0.index, int(0.8*len(X0)))
X0_80 = X0.ix[rows]
X0_20 = X0.drop(rows)
y0_80 = y0[:int(0.8*len(X0))]
y0_20 = y0[int(0.8*len(X0)):]
###

class_label = 1
train_neg = pd.DataFrame(neg_tweets, columns = ['text'])
X1 = train_neg.text
y1 = np.full(shape=len(X1), fill_value=class_label)

###create random 80-20 split
rows = random.sample(X1.index, int(0.8*len(X1)))
X1_80 = X1.ix[rows]
X1_20 = X1.drop(rows)
y1_80 = y1[:int(0.8*len(X1))]
y1_20 = y1[int(0.8*len(X1)):]
###

X_train = pd.concat((X0_80,X1_80))
y_train = np.concatenate((y0_80, y1_80))

X_holdout = pd.concat((X0_20,X1_20))
y_holdout = np.concatenate((y0_20, y1_20))

### load testing data

###read tagged sentiment Tweets
with open('data/'+suffix+'pos_examples_happy.txt', 'r') as f:
    pos_tweets = f.readlines()

with open('data/'+suffix+'neg_examples_sad.txt', 'r') as f:
    neg_tweets = f.readlines()
    
import numpy as np
class_label = 0
train_pos = pd.DataFrame(pos_tweets, columns = ['text'])
X0 = train_pos.text
y0 = np.full(shape=len(X0), fill_value=class_label)

class_label = 1
train_neg = pd.DataFrame(neg_tweets, columns = ['text'])
X1 = train_neg.text
y1 = np.full(shape=len(X1), fill_value=class_label)

X_test = pd.concat((X0,X1))
y_test = np.concatenate((y0, y1))

###
print "Loading data = COMPLETE!"

In [28]:
### build pipeline; fit on BEST CV MODEL train; predict on test!
import nltk
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import KFold

##vectorizer arguments blah!

tokenizer=None#word_tokenize
stop_words=nltk.corpus.stopwords.words("english")#None
ngram_range=(1, 3)
lowercase=True
max_features=5000
binary=False
dtype=np.float64
    
###create vectorizer
vectorizer = CountVectorizer(decode_error="ignore",
#                                tokenizer=tokenizer,
                               stop_words=stop_words,
                               ngram_range=ngram_range,
                               lowercase=lowercase,
                               binary=binary,
                               dtype=dtype,
                               max_features=max_features)

###5-fold cross validation
n_folds = 5
kf = KFold(len(X_train), n_folds=n_folds, shuffle=True, random_state=42)

###create pipeline
classifier = "MNB"
text_clf = Pipeline([
                     ('vect', vectorizer),
                     ('clf', MultinomialNB(alpha=0.1)),
#                     ('clf', LinearSVC(C=1.0)),
#         ("clf", RandomForestClassifier(n_estimators=100, max_features="auto", criterion="entropy", n_jobs=-1)),
                    ])


count = 0
max_fscore = 0.0
best_X_train = X_train
best_y_train = y_train
best_model = 0

for train, test in kf:
    count = count + 1
    print "Fold ", count
    X_CVtrain, y_CVtrain, X_CVtest, y_CVtest = X_train.values[train], y_train[train], X_train.values[test], y_train[test]
    ###fit training data
    text_clf = text_clf.fit(X_CVtrain, y_CVtrain)
    ### predict on test data
    predicted = text_clf.predict(X_CVtest)

    ###get performance metrics
    from sklearn.metrics import accuracy_score, precision_recall_fscore_support
    accuracy = accuracy_score(y_CVtest,predicted)
    precision, recall, fscore, sup = precision_recall_fscore_support(y_CVtest, predicted, average='binary', pos_label=0)
    print classifier, accuracy, precision, recall, fscore
    
    ###update best-model
    if max_fscore < fscore: 
        max_fscore = fscore
        best_X_train = X_CVtrain
        best_y_train = y_CVtrain
        best_X_test = X_CVtest
        best_y_test = y_CVtest
        best_model = count - 1
        
###fit the best NB model
NB_best = text_clf.fit(best_X_train, best_y_train)
###

###create pipeline
classifier = "LinearSVC"
text_clf = Pipeline([
                     ('vect', vectorizer),
#                      ('clf', MultinomialNB(alpha=0.1)),
                    ('clf', LinearSVC(C=1.0)),
#         ("clf", RandomForestClassifier(n_estimators=100, max_features="auto", criterion="entropy", n_jobs=-1)),
                    ])


count = 0
max_fscore = 0.0
best_X_train = X_train
best_y_train = y_train
best_model = 0

for train, test in kf:
    count = count + 1
    print "Fold ", count
    X_CVtrain, y_CVtrain, X_CVtest, y_CVtest = X_train.values[train], y_train[train], X_train.values[test], y_train[test]
    ###fit training data
    text_clf = text_clf.fit(X_CVtrain, y_CVtrain)
    ### predict on test data
    predicted = text_clf.predict(X_CVtest)

    ###get performance metrics
    from sklearn.metrics import accuracy_score, precision_recall_fscore_support
    accuracy = accuracy_score(y_CVtest,predicted)
    precision, recall, fscore, sup = precision_recall_fscore_support(y_CVtest, predicted, average='binary', pos_label=0)
    print classifier, accuracy, precision, recall, fscore
    
    ###update best-model
    if max_fscore < fscore: 
        max_fscore = fscore
        best_X_train = X_CVtrain
        best_y_train = y_CVtrain
        best_X_test = X_CVtest
        best_y_test = y_CVtest
        best_model = count - 1
        
###fit the best SVC model
SVC_best = text_clf.fit(best_X_train, best_y_train)

###

###create pipeline
classifier = "RandomForest"
text_clf = Pipeline([
                     ('vect', vectorizer),
#                      ('clf', MultinomialNB(alpha=0.1)),
#                     ('clf', LinearSVC(C=1.0)),
        ("clf", RandomForestClassifier(n_estimators=100, max_features="auto", criterion="entropy", n_jobs=-1)),
                    ])


count = 0
max_fscore = 0.0
best_X_train = X_train
best_y_train = y_train
best_model = 0

for train, test in kf:
    count = count + 1
    print "Fold ", count
    X_CVtrain, y_CVtrain, X_CVtest, y_CVtest = X_train.values[train], y_train[train], X_train.values[test], y_train[test]
    ###fit training data
    text_clf = text_clf.fit(X_CVtrain, y_CVtrain)
    ### predict on test data
    predicted = text_clf.predict(X_CVtest)

    ###get performance metrics
    from sklearn.metrics import accuracy_score, precision_recall_fscore_support
    accuracy = accuracy_score(y_CVtest,predicted)
    precision, recall, fscore, sup = precision_recall_fscore_support(y_CVtest, predicted, average='binary', pos_label=0)
    print classifier, accuracy, precision, recall, fscore
    
    ###update best-model
    if max_fscore < fscore: 
        max_fscore = fscore
        best_X_train = X_CVtrain
        best_y_train = y_CVtrain
        best_X_test = X_CVtest
        best_y_test = y_CVtest
        best_model = count - 1
        
###fit the best RF model
RF_best = text_clf.fit(best_X_train, best_y_train)

Fold  1
MNB 0.72890625 0.722150219095 0.737117298727 0.72955700346
Fold  2
MNB 0.724 0.738138686131 0.701734820322 0.719476559522
Fold  3
MNB 0.73446875 0.727764792717 0.744010564045 0.735798016231
Fold  4
MNB 0.72784375 0.743854344683 0.701655403311 0.72213891459
Fold  5
MNB 0.735 0.730964153276 0.741318791526 0.736105060061


In [29]:
best_model

4

In [30]:
print "validate best model"
text_clf = text_clf.fit(best_X_train, best_y_train)
### predict on test data
predicted = text_clf.predict(best_X_test)

###get performance metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
accuracy = accuracy_score(best_y_test,predicted)
precision, recall, fscore, sup = precision_recall_fscore_support(best_y_test, predicted, average='binary', pos_label=0)
print classifier, accuracy, precision, recall, fscore

validate best model
MNB 0.735 0.730964153276 0.741318791526 0.736105060061
